# Sentiment Analysis Project

## Mohammad Javad Maheronnaghsh

# Libraries

In [1]:
import pandas as pd
import requests
from io import StringIO
import numpy as np

# Data

In [2]:
link = 'https://drive.google.com/u/0/uc?id=1BRhlDz-JASvfEljf_XxrL-ZH4gUvOg0v&export=download'
r = requests.get(link)

In [3]:
df = pd.read_csv(StringIO(r.content.decode('utf-8')))

In [4]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
155043,155044,8485,Santa Clause 2 ' is wondrously creative .,4
155044,155045,8485,' is wondrously creative .,2
155045,155046,8485,is wondrously creative .,3
155046,155047,8485,is wondrously creative,4


# Data Split

In [5]:
y = df['Sentiment']
x = df.drop('Sentiment', axis = 1)

In [6]:
print(x.shape)
print(y.shape)

(155048, 3)
(155048,)


In [7]:
# Train/Validation/Test = 0.8/0.1/0.1
# stratify or not?
from sklearn.model_selection import train_test_split
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.2)
x_valid, x_test, y_valid, y_test = train_test_split(x_, y_, test_size=0.5)

# Preprocessing
Here we are going to preprocess the data
and extract the features

## First Apperoach: Basic Tools

What to do?


1.   Convert Capital Letters to Little Ones (English words)
2.   Delete Punctuations, etc.
3.   Delete Stop Words
4.   Find Origination of words
5.   Provide a list of remaind words
6.   Find TF-IDF vector

**Note: Weight of each token should be determined using train data**


In [8]:
# ! pip install fasttext

In [9]:
# ! git clone git@github.com:f20500909/glove_py.git
# ! python3 setup.py install

In [10]:
# import numpy as np
# import fasttext
# from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
# # from glove import *

# # from glove import Corpus, Glove
# # GloVe
# corpus = Corpus()
# corpus.fit(x_train, window=10)
# glove_model = Glove(no_components=100, learning_rate=0.05)
# glove_model.fit(corpus.matrix, epochs=10)
# glove_model.add_dictionary(corpus.dictionary)
# glove_vectors = []
# for doc in x_train:
#     vec = np.zeros(glove_model.no_components)
#     for word in doc.split():
#         if word in glove_model.dictionary:
#             vec += glove_model.word_vectors[glove_model.dictionary[word]]
#     glove_vectors.append(vec)

## Second Approach: Using Ready Tools

What to do?
We want to use one of the following models:
1.  Word2Vec
2.  FastText
3.  GloVe

These are the models that are frequently used in NLP areas for deature extraction from words and statements.

In [11]:
# install library
! pip install fasttext

In [12]:
x.shape

(155048, 3)

In [13]:
type(x)

pandas.core.frame.DataFrame

In [14]:
# FastText
import fasttext

def create_file(input, file_name):
  preprocessed_sentences = []
  for index, row in input.iterrows():
      phrase = row['Phrase']

      # Split by space
      tokens = phrase.split()

      # Filter out non-alphabetic tokens
      filtered_tokens = [token for token in tokens if token.isalpha()]
      if len(filtered_tokens) > 0:
        preprocessed_sentences.append(" ".join(filtered_tokens))
      else:
        preprocessed_sentences.append('random text')

  # Save preprocessed sentences to a text file
  with open(file_name, 'w', encoding='utf-8') as f:
      f.write('\n'.join(preprocessed_sentences))

create_file(x_train, 'train.txt')
create_file(x_valid, 'val.txt')
# Train FastText word embeddings
# model = fasttext.train_unsupervised('preprocessed.txt', model='cbow')

# # Save word vectors to a file
# model.save_model('fasttext_model.bin')


In [15]:
# # TF-IDF vectorization
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer()
# x_tfidf = vectorizer.fit_transform(preprocessed_sentences)
# remaining_tokens = vectorizer.get_feature_names()

# for token in remaining_tokens:
#         vector = model.get_word_vector(token)
#         vector_str = ' '.join(str(value) for value in vector)
#         line = f'{token} {vector_str}\n'
#         f.write(line)
# Get word vectors



# word_vectors = []
# for sentence in preprocessed_sentences:
#     tokens = sentence.split()
#     sentence_vector = np.zeros(model.get_dimension())
#     for token in tokens:
#         sentence_vector += model.get_word_vector(token)
#     sentence_vector /= len(tokens)
#     word_vectors.append(sentence_vector)

# # Convert word vectors to numpy array (the target vector)
# sentence_vectors = np.array(word_vectors)

# Train the Models

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import fasttext

# Download the required resources from NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Preprocessing Steps
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

x_train['Phrase'] = x_train['Phrase'].apply(preprocess_text)
x_valid['Phrase'] = x_valid['Phrase'].apply(preprocess_text)
x_test['Phrase'] = x_test['Phrase'].apply(preprocess_text)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
x_train.shape

(124038, 3)

In [18]:
train = np.hstack((x_train, y_train.values.reshape(-1, 1)))

In [19]:
df = pd.DataFrame(train, columns=['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'])

In [20]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,60468,3053,mediocrity,1
1,14100,608,compelling allegory last days germany democrat...,3
2,22642,1024,showcases,2
3,136130,7356,write,2
4,85981,4449,interesting look life campaigntrail press espe...,3
...,...,...,...,...
124033,59276,2990,cho may intended,2
124034,55694,2785,derivative collection,2
124035,146573,7972,nitpicky hypocrisies time,1
124036,99730,5232,look positively shakesperean comparison,2


In [27]:
# Training data
train_sentences = x_train['Phrase'].tolist()
train_labels = y_train.tolist()

# Validation data
val_sentences = x_valid['Phrase'].tolist()
val_labels = y_valid.tolist()

# Test data
test_sentences = x_test['Phrase'].tolist()
test_labels = y_test.tolist()

# Generate TF-IDF vectors using scikit-learn's TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_sentences)
X_val_tfidf = vectorizer.transform(val_sentences)
X_test_tfidf = vectorizer.transform(test_sentences)



df.to_csv('preprocessed_train_dataset.csv', index=False, header=False, columns=['Phrase'])

# Training data
train_data_path = 'preprocessed_train_dataset.csv'


# Prepare training data in FastText format
train_data = ['__label__' + str(label) + ' ' + sentence for label, sentence in zip(train_labels, train_sentences)]

# Save the training data to a text file
train_data_path = 'preprocessed_train_data.txt'
with open(train_data_path, 'w') as f:
    for item in train_data:
        f.write("%s\n" % item)


val_labels = ['__label__' + str(x) for x in val_labels]
test_labels = ['__label__' + str(x) for x in test_labels]

best_accuracy = 0
best_model = None

for lr in [0.05, 0.1]:
  for epoch in [10, 30]:
    for dim in [10, 100]:
      # Train FastText model
      ft_model = fasttext.train_supervised(
          input=train_data_path,
          epoch=epoch,
          lr=lr,
          wordNgrams=2,
          bucket=200000,
          dim=dim
      )

      # Predict on validation data
      val_predictions = [ft_model.predict(text)[0][0] for text in val_sentences]

      # Calculate accuracy on validation data

      val_accuracy = sum([val_labels[i] == val_predictions[i] for i in range(len(val_labels))]) / len(val_labels)
      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = ft_model



# Evaluate the Models

In [28]:
# Predict on test data
test_predictions = [best_model.predict(text)[0][0] for text in test_sentences]

# Calculate accuracy on test data
test_accuracy = sum([test_labels[i] == test_predictions[i] for i in range(len(test_labels))]) / len(test_labels)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.654627539503386
